## Stubborn Mining ##


#### In  this  section,  we  introduce  a  new mining strategy, called stubborn mining, that strictly generalizes (and improves upon) the prior known selfish mining attacker, causing the rest of the network to waste its hashpower on  redundant  blocks.  The  selfish  mining  strategy,in  particular,  withholds  blocks  when  it  is  “in  the  lead” (when  it  has  created  a  private  chain  longer  than  that of  the  honest  network),  but  cooperates  with  the  honest network  when  it  falls  behind.  The  key  insight  behind  stubborn  mining  is  that the  attacker  should not give up so easily! Instead, the attacker can often increase profits  by  mining  on  its  private  chain  more  often,  even under circumstances where a selfish-mining attacker would succumb to the public chain. There are two types of state transitions in this state machine:

* State  transitions  that  take  place  when  honest miner  finds  a  newblock. In this case, honest miner always announces the new block immediately.  At  this  point,  selfish miner  has  the  freedom  of performing  the  following  actions: 
    * If  selfish miner's  private chain is leading, Alice can decide whether and how many blocks  to  reveal  from  its  private  chain  to  Bob;  and  
    * If selfish miner's  private  chain  is  sufficiently  behind honest miner's  chain, selfish miner may choose to abandon mining on its private chain and accept honest miner's chain. selfish miner's decision in this case will define  how  the   chain  will  transition  when  honest miner mines a next block.
    
* State  transitions  that  take  place  when  selfish miner  finds  a  newblock.  In  the  latter  case,  when  selfish miner  mines  a  nextblock, we assume that selfish miner may simply continue to mine on  its  own  private  chain.  Whether  selfish miner  immediately reveals  her  new  block  to  honest miner  depends  on  the  strategy and the current state.


## Defining  Key  Parameters ##

### *   Hashpower  of  attacker (alpha) "α" :  The  fraction  of the  total     hashpower controlled by the attacker. 


### *   Network  influence (gamma) "γ" : fraction of honest miner's network(in  terms  of  hashpower)  that  will  mine  on  selfish miner's block  when both honest and selfish miners release a block at (approximately) the same time resulting in an equal length fork.
 


# Stubborn mining Strategies Simulated 

### As the basics of stubborn mining has been explained, we will simulate two (2) intriguing context namely "Equal Fork Stubborn Mining" and "Lead Stubborn Mining"

## Equal Fork Stubborn Mining ##

### Equal Fork stubborn (F-stubborn): When selfish miner wins a  race  in  state "lead=0′",  the  selfish-miner   would hurry to reveal her new block to the public, transitioning to "lead=0".  By  contrast,  the F-stubborn  miner  would conceal  her  new  block  and  continue  mining  on  it  privately,  thus  proceeding  to  state "lead=1".  In  Figure  5,this  event  corresponds  to  the  dotted  blue  edge  (labeled with α) leaving state 0′ into state 1.
###

<img src="Capture.jpg">

## Running a simulation for Equal Fork Stubborn Mining

In [59]:
import random
class StubbornMining:
    def __init__(self, p_chain, h_chain, s_blocks, h_blocks, s_orphans, h_orphans, up_blocks):
        self.p_chain = p_chain #selfish miners private chain length
        self.h_chain = h_chain  # public chain length
        self.s_blocks = s_blocks #published blocks mined by selfish miners
        self.h_blocks = h_blocks #blocks mined by honest miners
        self.s_orphans = s_orphans #selfish miners orphan blocks
        self.h_orphans = h_orphans #honest miners orphan blocks 
        self.up_blocks = up_blocks #unpublished blocks

def SimulateEqualForkStubborn(alpha, gamma, iteration):
    P = StubbornMining(0, 0, 0, 0, 0, 0, 0)
    state = 0

    for i in range(1,iteration):
        r = random.random()
        if state == 0:
            if r<alpha: #stubborn miner mined a block
                state = 1
                P.p_chain += 1
                P.up_blocks += 1
            else:
                P.h_blocks += 1

        elif state == 1:
            if r<alpha:
                state = 2
                P.p_chain += 1
                P.up_blocks += 1
            else:
                state = -1
                P.h_blocks += 1
                P.h_chain += 1
                P.up_blocks = 0
                P.s_blocks += 1 
        elif state == 2:
            if r<alpha:
                state = 3
                P.p_chain += 1
                P.up_blocks += 1
            else:
                state = -2
                P.up_blocks -= 1
                P.h_chain += 1
                P.s_blocks += 1 
                P.h_blocks += 1 
        elif state > 2:
            if r<alpha:
                state += 1
                P.p_chain += 1
                P.up_blocks += 1
            else:
                state = -state
                P.up_blocks -= 1
                P.h_chain += 1
                P.s_blocks += 1 
                P.h_blocks += 1 
        elif state == -1: #state zero prime
                if r<alpha:
                    state = -2
                    P.up_blocks += 1
                    P. p_chain += 1
                elif r<=(1-alpha)*(1-gamma):
                    state = 0
                    P.up_blocks = 0
                    P.p_chain = 0
                    P.h_chain = 0
                    P.h_blocks += 1
                    P.s_orphans += 1
                else:
                    state = 0
                    P.h_blocks += 1
                    P.h_orphans += 1
                    P.p_chain = 0
                    P.h_chain = 0
                    P.up_blocks = 0
        elif state == -2: #state one prime
                if r<alpha:
                        state -= 1
                        P.up_blocks += 1
                        P.p_chain += 1
                elif r<=(1-alpha)*(1-gamma):
                        state = -1
                        P.h_chain += 1
                        P.h_blocks += 1 
                        P.s_blocks += 1 
                        P.up_blocks -= 1
                else:
                        state = -1
                        P.up_blocks = 0
                        P.p_chain = 1
                        P.public_chain = 1
                        P.h_blocks += 1 
                        P.s_blocks += 1 
                        P.h_orphans += 1
        elif state<-2:
                if r<alpha:
                        state -= 1
                        P.p_chain += 1
                        P.up_blocks += 1
                elif r<=(1-alpha)*(1-gamma):
                        state += 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
                        P.up_blocks -= 1
                        P.h_chain += 1
                else:
                        state += 1
                        P.up_blocks -= 1
                        P.h_orphans += 1
                        P.h_chain = 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
    return P

In [60]:
def main():
    alpha = 0.33
    gamma = 0.3
    iteration = 10000
    P = SimulateEqualForkStubborn(alpha, gamma, iteration)

    print("\n Iterationations | %d \n alpha | %f (Selfish miner hash power) \n gamma | %f (Proportion of honest miners which mine on the selfish pool)" % (iteration, alpha, gamma))
    print("\n Theoretical Performance | %f \n Simulated Performance | %f" % ((alpha*(1-alpha)**2*(4*alpha+gamma*(1-2*alpha))-alpha**3)/(1-alpha*(1+(2-alpha)*alpha)), P.s_blocks / float(P.s_blocks + P.h_blocks)))
    print("\n Selfish Blocks Mined | %d \n Honest Blocks Mined | %d \n (Selfish Blocks Mined)/(Total Blocks Mined) | %f" % (P.s_blocks, P.h_blocks, P.s_blocks / float(P.s_blocks + P.h_blocks)))
    print("\n Selfish Orphan Blocks | %d \n Honest Orphan Blocks |  %d" % (P.s_orphans, P.h_orphans))
    print(" Difficulty | %d percent" % (float(P.s_blocks + P.h_blocks)/iteration*100))

    print(P.s_blocks)
    print(P.h_blocks)
    print(P.p_chain)
    print(P.up_blocks)
    print(P.h_orphans)
    print(P.s_orphans)



if __name__ == "__main__":
    main()



 Iterationations | 10000 
 alpha | 0.330000 (Selfish miner hash power) 
 gamma | 0.300000 (Proportion of honest miners which mine on the selfish pool)

 Theoretical Performance | 0.357920 
 Simulated Performance | 0.336401

 Selfish Blocks Mined | 3363 
 Honest Blocks Mined | 6634 
 (Selfish Blocks Mined)/(Total Blocks Mined) | 0.336401

 Selfish Orphan Blocks | 208 
 Honest Orphan Blocks |  2650
 Difficulty | 99 percent
3363
6634
3
2
2650
208


## Stubborn Mining for lead Fork ##

###  Lead-stubborn, outperforms selfish mining.  In  lead-stubborn  mining,instead  of selfiish miner revealing  its  entire  private  chain,  it  reveals the next block on her private chain only, to match the length of the public chain. In this case, "γ" fraction of honest miner will min eon selfish miner's private chain, and "1−γ" fraction mines on honest miner's fork; and the state transitions to "lead= 1′" per the code. This has pros and cons  for  the  selfish miner: 

*   if  the (1−γ) fraction  of  honest miner succeeds  in  advancing  on its  chain,  selfish miner  may  risk  losing her private chain. However, if selfish miner or the "γ" fraction of honest miner advances selfish miner's fork, then selfish miner has successfully diverted a part of honest miner, (1−γ) fraction, to do useless work. More generally, we could consider a more sophisticated lead-stubborn  miner  whose  action  (including  whether  to reveal  her  private  chain  and  how  many  blocks  to  reveal) depends  on  the  value  of lead -  state-dependent  actions. Our  simulation  results  suggest  that  even  without  considering state-dependent actions, a simple-minded lead-stubborn-miner  already  achieves  up  to  13.94%  higher  gains  than selfish  miner  under  typical  parameterizations  of "α" and "γ".


<img src="Capture1.png">

In [61]:
import random
class StubbornMining:
    def __init__(self, p_chain, h_chain, s_blocks, h_blocks, s_orphans, h_orphans, up_blocks):
        self.p_chain = p_chain #selfish miners private chain length
        self.h_chain = h_chain  #length of the public chain
        self.s_blocks = s_blocks #number of published blocks mined by selfish miners
        self.h_blocks = h_blocks #number of blocks mined by honest miners
        self.s_orphans = s_orphans #selfish miners orphan blocks 
        self.h_orphans = h_orphans #honest miners orphan blocks 
        self.up_blocks = up_blocks #unpublished blocks

def SimulateLeadStubborn(alpha, gamma, iteration):
    P = StubbornMining(0, 0, 0, 0, 0, 0, 0)
    state = 0

    for i in range(1,iteration):
        r = random.random()
        if state == 0:
            if r<alpha: #stubborn miner mined a block
                state = 1
                P.p_chain += 1
                P.up_blocks += 1
            else:
                P.h_blocks += 1

        elif state == 1:
            if r<alpha:
                state = 2
                P.p_chain += 1
                P.up_blocks += 1
            else:
                state = -1
                P.h_blocks += 1
                P.h_chain += 1
                P.up_blocks = 0
                P.s_blocks += 1 
        elif state == 2:
                if r<alpha:
                        state = 3
                        P.p_chain += 1
                        P.up_blocks += 1
                else:
                        state = -2
                        P.up_blocks -= 1
                        P.h_chain += 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
        elif state > 2:
                if r<alpha:
                        state += 1
                        P.p_chain += 1
                        P.up_blocks += 1
                else:
                        state = -state
                        P.up_blocks -= 1
                        P.h_chain += 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
        elif state == -1: #state zero prime
                state = 0
                if r<alpha:
                        P.up_blocks = 0
                        P. p_chain = 0
                        P.h_chain = 0
                        P.s_blocks += 1
                        P.h_orphans += 1
                elif r<=alpha+(1-alpha)*gamma:
                        P.up_blocks = 0
                        P.p_chain = 0
                        P.h_chain = 0
                        P.h_blocks += 1
                        P.s_orphans += 1
                else:
                        P.h_blocks += 1
                        P.h_orphans += 1
                        P.p_chain = 0
                        P.h_chain = 0
                        P.up_blocks = 0
        elif state == -2: #state one prime
                if r<alpha:
                        state -= 1
                        P.up_blocks += 1
                        P.p_chain += 1
                elif r<=(1-alpha)*(1-gamma):
                        state = -1
                        P.h_chain += 1
                        P.h_blocks += 1 
                        P.s_blocks += 1 
                        P.up_blocks -= 1
                else:
                        state = -1
                        P.up_blocks = 0
                        P.p_chain = 1
                        P.public_chain = 1
                        P.h_blocks += 1 
                        P.s_blocks += 1 
                        P.h_orphans += 1
        elif state<-2:
                if r<alpha:
                        state -= 1
                        P.p_chain += 1
                        P.up_blocks += 1
                elif r<=(1-alpha)*(1-gamma):
                        state += 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
                        P.up_blocks -= 1
                        P.h_chain += 1
                else:
                        state += 1
                        P.up_blocks -= 1
                        P.h_orphans += 1
                        P.h_chain = 1
                        P.s_blocks += 1 
                        P.h_blocks += 1 
    return P

def main():
    alpha = 0.33
    gamma = 0.3
    iteration = 10000
    P = SimulateLeadStubborn(alpha, gamma, iteration)

    print("\n Iterationations | %d \n alpha | %f (Selfish miner hash power) \n gamma | %f (Proportion of honest miners which mine on the selfish pool)" % (iteration, alpha, gamma))
    print("\n Theoretical Performance | %f \n Simulated Performance | %f" % ((alpha*(1-alpha)**2*(4*alpha+gamma*(1-2*alpha))-alpha**3)/(1-alpha*(1+(2-alpha)*alpha)), P.s_blocks / float(P.s_blocks + P.h_blocks)))
    print("\n Selfish Blocks Mined | %d \n Honest Blocks Mined | %d \n (Selfish Blocks Mined)/(Total Blocks Mined) | %f" % (P.s_blocks, P.h_blocks, P.s_blocks / float(P.s_blocks + P.h_blocks)))
    print("\n Selfish Orphan Blocks | %d \n Honest Orphan Blocks |  %d" % (P.s_orphans, P.h_orphans))
    print(" Difficulty | %d percent" % (float(P.s_blocks + P.h_blocks)/iteration*100))

    print(P.s_blocks)
    print(P.h_blocks)
    print(P.p_chain)
    print(P.up_blocks)
    print(P.h_orphans)
    print(P.s_orphans)



if __name__ == "__main__":
    main()



 Iterationations | 10000 
 alpha | 0.330000 (Selfish miner hash power) 
 gamma | 0.300000 (Proportion of honest miners which mine on the selfish pool)

 Theoretical Performance | 0.357920 
 Simulated Performance | 0.329832

 Selfish Blocks Mined | 3297 
 Honest Blocks Mined | 6699 
 (Selfish Blocks Mined)/(Total Blocks Mined) | 0.329832

 Selfish Orphan Blocks | 308 
 Honest Orphan Blocks |  2238
 Difficulty | 99 percent
3297
6699
5
3
2238
308
